# 1. Mô hình Dự đoán Thời gian Khởi hành và Đến Nơi

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

# Đọc dữ liệu từ tệp CSV
data_frame = pd.read_csv('cleaned_strains.csv')

# Chuyển đổi cột giờ thành phút một cách tối ưu hơn
data_frame['Arrival_Minutes'] = (
    data_frame['Properties Arrival'].str.split(':', expand=True).astype(int).apply(lambda x: x[0] * 60 + x[1], axis=1)
)

# Chọn các cột cần thiết cho mô hình
features = data_frame[['Properties Chair Car', 'Properties First Class', 'Properties Sleeper', 
                       'Properties Duration M', 'Properties Duration H', 'Properties Distance']]
labels = data_frame['Arrival_Minutes']  # Để dự đoán thời gian đến nơi

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
features_train, features_test, labels_train, labels_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

# Chuẩn hóa dữ liệu
data_scaler = StandardScaler()
features_train = data_scaler.fit_transform(features_train)
features_test = data_scaler.transform(features_test)

# Xây dựng mô hình với Regularization và Dropout
model_arrival = keras.Sequential([
    layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01), input_shape=(features_train.shape[1],)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(1)  # Đầu ra là một giá trị
])

model_arrival.compile(optimizer='adam', loss='mean_squared_error')

# Sử dụng Early Stopping để tránh overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Huấn luyện mô hình
model_arrival.fit(features_train, labels_train, epochs=50, validation_split=0.2, callbacks=[early_stopping])

# Lưu mô hình
model_arrival.save('arrival_model.keras')


Epoch 1/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 763589.1875 - val_loss: 719529.3750
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 714648.0625 - val_loss: 534050.6250
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 473841.8750 - val_loss: 227616.4531
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 209293.8750 - val_loss: 121516.2969
Epoch 5/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 134974.9844 - val_loss: 116731.5391
Epoch 6/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 135719.3594 - val_loss: 115345.1094
Epoch 7/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 134162.9062 - val_loss: 114331.7109
Epoch 8/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 131622.3906 - val_loss: 113290.7891
Epoch 9/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 127619.2891 - val_loss: 112492.9219
Epoch 10/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 124256.2031 - val_loss: 111769.5156
Epoch 11/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 

# 2. Mô hình Phân loại Loại Tàu

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

# Đọc dữ liệu từ tệp CSV
data_frame = pd.read_csv('cleaned_strains.csv')

# Giả định rằng có cột 'Properties Type' chứa loại tàu
# Mã hóa loại tàu
label_encoder = LabelEncoder()
data_frame['Train_Type'] = label_encoder.fit_transform(data_frame['Properties Type'])

# Chọn các cột cần thiết cho mô hình phân loại
features_classification = data_frame[['Properties Chair Car', 'Properties First Class', 'Properties Sleeper']]
labels_classification = data_frame['Train_Type']  # Đối với phân loại

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
features_train_class, features_test_class, labels_train_class, labels_test_class = train_test_split(
    features_classification, 
    labels_classification, 
    test_size=0.2, 
    random_state=42
)

# Xây dựng mô hình phân loại
model_type = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(features_train_class.shape[1],)),
    layers.Dropout(0.5),  # Giảm thiểu overfitting
    layers.Dense(64, activation='relu'),
    layers.Dense(len(np.unique(labels_classification)), activation='softmax')  # Số lớp bằng số loại tàu
])

model_type.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Thiết lập EarlyStopping để dừng huấn luyện sớm
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Huấn luyện mô hình
model_type.fit(features_train_class, labels_train_class, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Lưu mô hình
model_type.save('train_type_model.keras')


Epoch 1/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.5226 - loss: 2.3254 - val_accuracy: 0.6735 - val_loss: 1.1530
Epoch 2/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6569 - loss: 1.1526 - val_accuracy: 0.6735 - val_loss: 0.9835
Epoch 3/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6662 - loss: 1.0300 - val_accuracy: 0.6844 - val_loss: 0.9616
Epoch 4/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6964 - loss: 0.9593 - val_accuracy: 0.6844 - val_loss: 0.9557
Epoch 5/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6825 - loss: 0.9886 - val_accuracy: 0.6844 - val_loss: 0.9479
Epoch 6/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6819 - loss: 0.9736 - val_accuracy: 0.6844 - val_loss: 0.9476
Epoch 7/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6682 - loss: 0.9745 - val_accuracy: 0.6844 - val_loss: 0.9462
Epoch 8/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6825 - loss: 0.9587 - val_accuracy: 0.6844 - v

# 3. Mô hình Dự đoán Doanh Thu

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

# Đọc dữ liệu từ tệp CSV
data_frame = pd.read_csv('cleaned_strains.csv')

# Giả định rằng có cột 'Revenue' chứa doanh thu
data_frame['Revenue'] = (data_frame['Properties Chair Car'] * 10 + 
                          data_frame['Properties First Class'] * 20 + 
                          data_frame['Properties Sleeper'] * 15)  # Giả định cách tính doanh thu

# Chọn các cột cần thiết cho mô hình dự đoán doanh thu
features_revenue = data_frame[['Properties Chair Car', 'Properties First Class', 'Properties Sleeper']]
labels_revenue = data_frame['Revenue']  # Cột doanh thu

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
features_train_rev, features_test_rev, labels_train_rev, labels_test_rev = train_test_split(
    features_revenue, labels_revenue, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
features_train_rev = scaler.fit_transform(features_train_rev)
features_test_rev = scaler.transform(features_test_rev)

# Xây dựng mô hình
model_revenue = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(features_train_rev.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Đầu ra là một giá trị doanh thu
])

model_revenue.compile(optimizer='adam', loss='mean_squared_error')

# Thiết lập Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Huấn luyện mô hình
model_revenue.fit(features_train_rev, labels_train_rev, epochs=100, validation_split=0.2, callbacks=[early_stopping])

# Lưu mô hình
model_revenue.save('revenue_model.keras')


Epoch 1/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 88.4357 - val_loss: 0.8259
Epoch 2/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7008 - val_loss: 0.1788
Epoch 3/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3151 - val_loss: 0.0326
Epoch 4/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0441 - val_loss: 8.2335e-04
Epoch 5/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0227 - val_loss: 2.9843e-04
Epoch 6/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0120 - val_loss: 3.4562e-04
Epoch 7/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0042 - val_loss: 6.0416e-05
Epoch 8/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0020 - val_loss: 1.5963e-04
Epoch 9/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013 - val_loss: 1.3281e-05
Epoch 10/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.8803e-04 - val_loss: 2.4635e-06
Epoch 11/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.4505e-05 - val_loss: 1.4879e-06
Epoch 12/100
92/92 ━━━

# 4. Mô hình Phân Tích Cảm Xúc

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

# Đọc dữ liệu từ tệp CSV với tối ưu hóa dtypes
data_frame = pd.read_csv('cleaned_strains.csv', dtype={
    'Properties Chair Car': 'int8',
    'Properties First Class': 'int8',
    'Properties Sleeper': 'int8'
})

# Giả định rằng có cột 'Sentiment' chứa cảm xúc
data_frame['Sentiment'] = np.random.choice(['Positive', 'Negative', 'Neutral'], size=len(data_frame))  # Giả định dữ liệu cảm xúc

# Mã hóa cảm xúc
label_encoder = LabelEncoder()
data_frame['Sentiment'] = label_encoder.fit_transform(data_frame['Sentiment'])

# Chọn các cột cần thiết cho mô hình phân tích cảm xúc
features_sentiment = data_frame[['Properties Chair Car', 'Properties First Class', 'Properties Sleeper']]
labels_sentiment = data_frame['Sentiment']  # Cột cảm xúc

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
features_train_sent, features_test_sent, labels_train_sent, labels_test_sent = train_test_split(
    features_sentiment, labels_sentiment, test_size=0.2, random_state=42
)

# Xây dựng mô hình với Dropout để ngăn ngừa overfitting
model_sentiment = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(features_train_sent.shape[1],)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(3, activation='softmax')  # Giả định có 3 lớp cảm xúc
])

model_sentiment.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Sử dụng EarlyStopping để ngăn chặn overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Huấn luyện mô hình
model_sentiment.fit(features_train_sent, labels_train_sent, epochs=50, validation_split=0.2, batch_size=32, callbacks=[early_stopping])

# Lưu mô hình
model_sentiment.save('sentiment_model.keras')


Epoch 1/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.3128 - loss: 1.1021 - val_accuracy: 0.3687 - val_loss: 1.0964
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3386 - loss: 1.0995 - val_accuracy: 0.3265 - val_loss: 1.0984
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3474 - loss: 1.1001 - val_accuracy: 0.3265 - val_loss: 1.0986
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3278 - loss: 1.0998 - val_accuracy: 0.3238 - val_loss: 1.0995
Epoch 5/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3440 - loss: 1.0986 - val_accuracy: 0.3619 - val_loss: 1.0984
Epoch 6/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3406 - loss: 1.0983 - val_accuracy: 0.3619 - val_loss: 1.0984


# 5. Mô hình Tối ưu hóa Lịch Trình

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

# Đọc dữ liệu từ tệp CSV và loại bỏ giá trị thiếu
data_frame = pd.read_csv('cleaned_strains.csv').dropna()

# Giả định rằng có cột 'Optimal_Schedule' chứa lịch trình tối ưu
data_frame['Optimal_Schedule'] = data_frame['Properties Duration H'] + 1  # Giả định một cách tối ưu hóa

# Chọn các cột cần thiết cho mô hình tối ưu hóa lịch trình
features_schedule = data_frame[['Properties Chair Car', 'Properties First Class', 'Properties Sleeper', 'Properties Duration M']]
labels_schedule = data_frame['Optimal_Schedule']  # Cột lịch trình tối ưu

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
features_train_sched, features_test_sched, labels_train_sched, labels_test_sched = train_test_split(
    features_schedule, labels_schedule, test_size=0.2, random_state=42
)

# Xây dựng mô hình với Batch Normalization và Dropout
model_schedule = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(features_train_sched.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(1)  # Đầu ra là một giá trị cho lịch trình tối ưu
])

model_schedule.compile(optimizer='adam', loss='mean_squared_error')

# Sử dụng EarlyStopping để tránh overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Huấn luyện mô hình
model_schedule.fit(
    features_train_sched, labels_train_sched,
    epochs=100, 
    validation_split=0.2,
    callbacks=[early_stopping],
    batch_size=32
)

# Lưu mô hình
model_schedule.save('schedule_model.keras')


Epoch 1/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 302.6790 - val_loss: 291.9205
Epoch 2/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 204.1088 - val_loss: 233.2955
Epoch 3/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 169.9693 - val_loss: 180.8554
Epoch 4/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 129.9129 - val_loss: 127.6384
Epoch 5/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 106.8174 - val_loss: 98.2868
Epoch 6/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 93.0599 - val_loss: 95.5288
Epoch 7/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 88.1337 - val_loss: 90.7213
Epoch 8/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 94.0554 - val_loss: 87.4800
Epoch 9/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 84.4581 - val_loss: 104.7879
Epoch 10/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 84.3177 - val_loss: 90.3540
Epoch 11/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 84.6916 - val_loss: 90.4234
Epoch 12/100
92/92 ━━━━━━━━━━━━

# 6. Mô hình Phát hiện Bất Thường

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

# Đọc dữ liệu từ tệp CSV
data_frame = pd.read_csv('cleaned_strains.csv')

# Giả định rằng có cột 'Anomaly' để phát hiện bất thường
data_frame['Anomaly'] = np.random.choice([0, 1], size=len(data_frame))  # Giả định có cột bất thường

# Chọn các cột cần thiết cho mô hình phát hiện bất thường
features_anomaly = data_frame[['Properties Chair Car', 'Properties First Class', 'Properties Sleeper', 'Properties Duration M']]
labels_anomaly = data_frame['Anomaly']  # Cột bất thường

# Xử lý dữ liệu: loại bỏ giá trị thiếu và chuẩn hóa
features_anomaly = features_anomaly.dropna()
scaler = StandardScaler()
features_anomaly_scaled = scaler.fit_transform(features_anomaly)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
features_train_anomaly, features_test_anomaly, labels_train_anomaly, labels_test_anomaly = train_test_split(
    features_anomaly_scaled, labels_anomaly[features_anomaly.index], test_size=0.2, random_state=42
)

# Xây dựng mô hình
input_layer = layers.Input(shape=(features_train_anomaly.shape[1],))
encoded = layers.Dense(64, activation='relu')(input_layer)  # Tăng số nơ-ron
encoded = layers.Dense(32, activation='relu')(encoded)  # Thêm lớp nơ-ron
decoded = layers.Dense(64, activation='relu')(encoded)  # Tăng số nơ-ron cho lớp giải mã
decoded = layers.Dense(features_train_anomaly.shape[1], activation='sigmoid')(decoded)

autoencoder = keras.Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Thêm EarlyStopping để ngăn chặn overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Huấn luyện mô hình
autoencoder.fit(features_train_anomaly, features_train_anomaly, epochs=100, validation_split=0.2, callbacks=[early_stopping])

# Lưu mô hình
autoencoder.save('anomaly_detection_model.keras')

# Đánh giá mô hình trên tập kiểm tra
reconstructed = autoencoder.predict(features_test_anomaly)
mse = np.mean(np.power(features_test_anomaly - reconstructed, 2), axis=1)
print("MSE on test set:", mse.mean())


Epoch 1/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 1.0660 - val_loss: 0.5750
Epoch 2/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5669 - val_loss: 0.5364
Epoch 3/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5218 - val_loss: 0.5342
Epoch 4/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5555 - val_loss: 0.5336
Epoch 5/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5155 - val_loss: 0.5334
Epoch 6/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5352 - val_loss: 0.5333
Epoch 7/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5643 - val_loss: 0.5332
Epoch 8/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5226 - val_loss: 0.5331
Epoch 9/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5154 - val_loss: 0.5331
Epoch 10/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5288 - val_loss: 0.5331
Epoch 11/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5511 - val_loss: 0.5330
Epoch 12/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5

# 7. Mô hình Dự đoán Thời gian Tới Đích

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Đọc dữ liệu từ tệp CSV
data_frame = pd.read_csv('cleaned_strains.csv')

# Chuyển đổi cột giờ thành phút
def time_to_minutes(time_string):
    hours, minutes, seconds = map(int, time_string.split(':'))
    return hours * 60 + minutes

# Chuyển đổi cột giờ thành phút
data_frame['Departure_Minutes'] = data_frame['Properties Departure'].apply(time_to_minutes)
# Chọn các cột cần thiết cho mô hình
features_destination = data_frame[['Properties Chair Car', 'Properties First Class', 'Properties Sleeper', 
                                    'Properties Duration M', 'Properties Duration H', 'Properties Distance']]
labels_destination = data_frame['Departure_Minutes']  # Để dự đoán thời gian tới đích

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
features_train_dest, features_test_dest, labels_train_dest, labels_test_dest = train_test_split(
    features_destination, labels_destination, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
features_train_dest = scaler.fit_transform(features_train_dest)
features_test_dest = scaler.transform(features_test_dest)

# Xây dựng mô hình
model_destination = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(features_train_dest.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Đầu ra là một giá trị
])

model_destination.compile(optimizer='adam', loss='mean_squared_error')

# Thiết lập các callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_destination_model.keras', save_best_only=True)

# Huấn luyện mô hình
model_destination.fit(features_train_dest, labels_train_dest, epochs=100, validation_split=0.2, 
                      batch_size=32, callbacks=[early_stopping, model_checkpoint])

# Đánh giá mô hình trên tập kiểm tra
test_loss = model_destination.evaluate(features_test_dest, labels_test_dest)
print(f'Test Loss: {test_loss}')

# Lưu mô hình cuối cùng
model_destination.save('final_destination_model.keras')


Epoch 1/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 745450.6250 - val_loss: 715321.6250
Epoch 2/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 710200.8750 - val_loss: 525378.6250
Epoch 3/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 455683.7188 - val_loss: 248551.0781
Epoch 4/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 215009.0625 - val_loss: 154839.2500
Epoch 5/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 153073.5469 - val_loss: 133832.1406
Epoch 6/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 133815.5469 - val_loss: 123580.0078
Epoch 7/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 124973.0234 - val_loss: 117864.0000
Epoch 8/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 121166.8750 - val_loss: 114783.1094
Epoch 9/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 117007.8359 - val_loss: 112682.3203
Epoch 10/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 115886.6953 - val_loss: 111537.8203
Epoch 11/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/st